In [14]:
# ... keep everything you already have above (imports, config, loaders, simulate_one_ticker, etc.) ...

import math
import pandas as pd
import yfinance as yf

# ---- Helpers for risk/return stats ----

def _max_drawdown_from_equity(equity: pd.Series) -> float:
    """Return MDD as a negative fraction, e.g., -0.18 for -18%."""
    if equity.empty:
        return 0.0
    roll_max = equity.cummax()
    dd = equity / roll_max - 1.0
    return float(dd.min()) if not dd.empty else 0.0

def _annualize_from_daily(mean_daily: float, std_daily: float) -> tuple[float, float]:
    """Return (annualized_mean, annualized_std) assuming 252 trading days."""
    ann_mean = (1 + mean_daily) ** 252 - 1
    ann_std = std_daily * math.sqrt(252)
    return ann_mean, ann_std

def _geom_daily_from_total(ret_total: float, days: int) -> float:
    """Per-day geometric rate so that (1+r)^days - 1 = ret_total."""
    days = max(int(days), 1)
    return (1.0 + ret_total) ** (1.0 / days) - 1.0

def _load_benchmark_series(start_date: str | None, end_date: str | None) -> pd.Series:
    b = yf.download("^NSEI", start=start_date, end=end_date, interval="1d",
                    auto_adjust=True, progress=False, multi_level_index=False)
    if b is None or b.empty:
        return pd.Series(dtype=float)

    price_col = "Adj Close" if "Adj Close" in b.columns else "Close"

    # Make index tz-aware -> IST, then collapse to trading date
    if b.index.tz is None:
        b.index = b.index.tz_localize("UTC")
    b = b.tz_convert("Asia/Kolkata")

    b = b.dropna(subset=[price_col]).copy()
    b["date"] = b.index.tz_localize(None).date
    b = b.groupby("date")[price_col].last()

    bench_ret = b.pct_change().dropna()
    bench_ret.index = pd.to_datetime(bench_ret.index)
    bench_ret.name = "bench_ret"
    return bench_ret



def _build_strategy_daily(trades_df: pd.DataFrame) -> pd.DataFrame:
    """
    Build average per-block daily return series from trades.
    Each trade i contributes a constant geometric daily return r_i over its holding days.
    On days with multiple trades, we average (not sum) to get a per-block return.
    """
    if trades_df.empty:
        return pd.DataFrame({"ret": pd.Series(dtype=float)})

    # Parse times as naive dates (already stored as YYYY-MM-DD in your script)
    df = trades_df.copy()
    df["entry_time"] = pd.to_datetime(df["entry_time"]).dt.normalize()
    df["exit_time"]  = pd.to_datetime(df["exit_time"]).dt.normalize()
    df["days_held"]  = df["days_held"].astype(int).clip(lower=1)

    # Per-trade total return (net)
    df["r_total"] = df["net_pnl_inr"] / (df["entry_price"] * df["shares"])
    df["r_total"] = df["r_total"].fillna(0.0)

    start = df["entry_time"].min()
    end   = df["exit_time"].max()
    if pd.isna(start) or pd.isna(end):
        return pd.DataFrame({"ret": pd.Series(dtype=float)})

    # Use business days between first entry and last exit
    idx = pd.bdate_range(start, end, freq="C")
    sum_ret = pd.Series(0.0, index=idx)
    act_cnt = pd.Series(0.0, index=idx)

    for _, r in df.iterrows():
        # Holding window: from next business day after entry to exit day (inclusive)
        # If you want to include entry day too, change start_day = r["entry_time"]
        start_day = r["entry_time"] + pd.offsets.BusinessDay(1)
        end_day   = r["exit_time"]
        if end_day < start_day:
            # Single-day trade: allocate on exit day
            start_day = end_day
        window = pd.bdate_range(start_day, end_day, freq="C")
        if window.empty:
            continue
        daily_r = _geom_daily_from_total(float(r["r_total"]), int(r["days_held"]))
        sum_ret.loc[window] += daily_r
        act_cnt.loc[window] += 1.0

    avg_ret = sum_ret.copy()
    mask = act_cnt > 0
    avg_ret[mask] = sum_ret[mask] / act_cnt[mask]
    avg_ret[~mask] = 0.0  # flat when no positions

    return pd.DataFrame({"ret": avg_ret})

def _up_down_capture(strat_ret: pd.Series, bench_ret: pd.Series) -> dict:
    """
    Geometric up/down capture using daily returns:
      - Up capture: annualized (geo) strategy return over bench up days / same for bench
      - Down capture: annualized (geo) strategy return over bench down days / same for bench
    """
    s = strat_ret.dropna()
    b = bench_ret.reindex_like(s).dropna()
    s = s.reindex(b.index)

    caps = {"up_capture_pct": float("nan"), "down_capture_pct": float("nan")}

    # Up days
    up = b > 0
    if up.any():
        s_up = (1 + s[up]).prod()
        b_up = (1 + b[up]).prod()
        # Annualize by number of up days
        n_up = int(up.sum())
        s_up_cagr = s_up ** (252.0 / n_up) - 1.0 if n_up > 0 else float("nan")
        b_up_cagr = b_up ** (252.0 / n_up) - 1.0 if n_up > 0 else float("nan")
        if b_up_cagr and not math.isclose(b_up_cagr, 0.0):
            caps["up_capture_pct"] = 100.0 * (s_up_cagr / b_up_cagr)

    # Down days
    down = b < 0
    if down.any():
        s_dn = (1 + s[down]).prod()
        b_dn = (1 + b[down]).prod()
        n_dn = int(down.sum())
        s_dn_cagr = s_dn ** (252.0 / n_dn) - 1.0 if n_dn > 0 else float("nan")
        b_dn_cagr = b_dn ** (252.0 / n_dn) - 1.0 if n_dn > 0 else float("nan")
        if b_dn_cagr and not math.isclose(b_dn_cagr, 0.0):
            caps["down_capture_pct"] = 100.0 * (s_dn_cagr / b_dn_cagr)

    return caps

def _trade_metrics(trades_df: pd.DataFrame) -> dict:
    """Trade-level metrics: win rate, payoff, expectancy, avg duration, profit factor."""
    if trades_df.empty:
        return {
            "trades": 0, "win_rate_pct": 0.0, "payoff_ratio": 0.0, "expectancy_pct": 0.0,
            "avg_trade_days": 0.0, "profit_factor": 0.0
        }
    df = trades_df.copy()
    df["ret_pct"] = 100.0 * df["net_pnl_inr"] / (df["entry_price"] * df["shares"])
    wins = df[df["ret_pct"] > 0]["ret_pct"]
    losses = df[df["ret_pct"] < 0]["ret_pct"]

    win_rate = 100.0 * (len(wins) / len(df)) if len(df) else 0.0
    avg_win  = wins.mean() if len(wins) else 0.0
    avg_loss = losses.mean() if len(losses) else 0.0  # negative
    payoff   = (avg_win / abs(avg_loss)) if avg_loss < 0 else float("inf") if avg_win > 0 else 0.0
    expectancy = (win_rate / 100.0) * avg_win + (1 - win_rate / 100.0) * avg_loss

    # Profit factor using INR (safer to compare rupees)
    gp = df[df["net_pnl_inr"] > 0]["net_pnl_inr"].sum()
    gl = -df[df["net_pnl_inr"] < 0]["net_pnl_inr"].sum()
    pf = (gp / gl) if gl > 0 else (float("inf") if gp > 0 else 0.0)

    avg_days = float(df["days_held"].mean()) if "days_held" in df.columns else 0.0
    return {
        "trades": int(len(df)),
        "win_rate_pct": float(win_rate),
        "payoff_ratio": float(payoff),
        "expectancy_pct": float(expectancy),  # per trade, in %
        "avg_trade_days": float(avg_days),
        "profit_factor": float(pf),
    }

def _portfolio_metrics(daily_ret: pd.Series) -> dict:
    """CAGR, MDD, Calmar, Sharpe, Sortino from average per-block daily returns."""
    r = daily_ret.fillna(0.0)
    if r.empty:
        return {
            "cagr_pct": 0.0, "mdd_pct": 0.0, "calmar": 0.0,
            "sharpe": 0.0, "sortino": 0.0, "exposure_pct": 0.0, "trading_days": 0
        }

    # Equity curve from average daily returns (start at 1.0)
    eq = (1.0 + r).cumprod()

    # Exposure: fraction of days where strategy had at least one active trade
    exposure = (r != 0).mean()

    # CAGR
    n = len(r)
    cagr = eq.iloc[-1] ** (252.0 / n) - 1.0 if n > 0 else 0.0

    # MDD (as positive %)
    mdd = abs(_max_drawdown_from_equity(eq))

    # Calmar
    calmar = (cagr / mdd) if mdd > 0 else float("inf") if cagr > 0 else 0.0

    # Sharpe & Sortino (rf ~ 0)
    mu = r.mean()
    sd = r.std(ddof=0)
    ann_mu, ann_sd = _annualize_from_daily(mu, sd)
    sharpe = (ann_mu / ann_sd) if ann_sd > 0 else float("inf") if ann_mu > 0 else 0.0

    downside = r[r < 0]
    dd_std = downside.std(ddof=0)
    sortino = (ann_mu / (dd_std * math.sqrt(252))) if dd_std > 0 else float("inf") if ann_mu > 0 else 0.0

    return {
        "cagr_pct": 100.0 * cagr,
        "mdd_pct": 100.0 * mdd,
        "calmar": float(calmar),
        "sharpe": float(sharpe),
        "sortino": float(sortino),
        "exposure_pct": 100.0 * exposure,
        "trading_days": int(n),
    }

def _turnover_stats(trades_df: pd.DataFrame, start_date: pd.Timestamp, end_date: pd.Timestamp) -> dict:
    """Turnover as trades/year and notional turnover per capital block."""
    if trades_df.empty or pd.isna(start_date) or pd.isna(end_date):
        return {"years": 0.0, "trades_per_year": 0.0, "roundtrip_turnover_per_year": 0.0}
    years = max((end_date - start_date).days / 365.25, 1e-9)
    trades = len(trades_df)
    tpy = trades / years
    # per-block notional turnover ≈ 2 * trades per year (buy+sell) relative to block capital
    return {
        "years": years,
        "trades_per_year": tpy,
        "roundtrip_turnover_per_year": 2.0 * tpy,
    }

def evaluate_strategy(trades_rows: list[dict], start_date: str | None, end_date: str | None) -> dict:
    """Compute all requested metrics and return a dict. Also returns daily series for optional saving."""
    trades_df = pd.DataFrame(trades_rows)
    if trades_df.empty:
        return {"metrics": {}, "daily": pd.Series(dtype=float)}

    # Core trade-level stats
    tmetrics = _trade_metrics(trades_df)

    # Build average per-block daily return series
    daily_df = _build_strategy_daily(trades_df)
    strat_daily = daily_df["ret"] if "ret" in daily_df else pd.Series(dtype=float)

    # Portfolio-level stats
    pmetrics = _portfolio_metrics(strat_daily)

    # Benchmark up/down capture (align to same days)
    bench = _load_benchmark_series(start_date, end_date)
    bench = bench.reindex(strat_daily.index)
    captures = _up_down_capture(strat_daily, bench)

    # Turnover (calendar years between first entry & last exit)
    sdate = pd.to_datetime(trades_df["entry_time"]).min()
    edate = pd.to_datetime(trades_df["exit_time"]).max()
    turn = _turnover_stats(trades_df, sdate, edate)

    # Merge all
    metrics = {}
    metrics.update(tmetrics)
    metrics.update(pmetrics)
    metrics.update(captures)
    metrics.update(turn)

    return {"metrics": metrics, "daily": strat_daily}

# ---- Update main() to compute & save metrics ----

def main():
    all_orders, all_trades = [], []
    for tk in UNIVERSE:
        bars = load_bars_for_ticker_yf(tk)
        if not bars or len(bars) < max(BB_WINDOW, ATR_LEN) + 5:
            continue
        o_rows, t_rows = simulate_one_ticker(
            tk, bars,
            allow_longs=ALLOW_LONGS,
            allow_shorts=ALLOW_SHORTS,
        )
        all_orders.extend(o_rows)
        all_trades.extend(t_rows)

    orders_header = ["trade_id","ticker","side","datetime","price","shares","reason","pnl_inr","pnl_pct"]
    trades_header = [
        "trade_id","ticker","entry_time","entry_price","exit_time","exit_price","shares",
        "days_held","exit_reason","fees_inr","gross_pnl_inr","net_pnl_inr","net_pnl_pct"
    ]

    # Save orders/trades
    write_csv(ORDERS_CSV, all_orders, orders_header)
    write_csv(TRADES_CSV, all_trades, trades_header)

    # ---- NEW: Evaluate strategy ----
    start = START_DATE
    end   = END_DATE
    eval_out = evaluate_strategy(all_trades, start, end)
    metrics = eval_out["metrics"]
    strat_daily = eval_out["daily"]

    # Print a tidy block
    print("Downloaded & simulated", len(UNIVERSE), "tickers.")
    print("Orders ->", ORDERS_CSV, "| Trades ->", TRADES_CSV)
    print("\n===== Strategy Metrics =====")
    for k in [
        "trades","win_rate_pct","payoff_ratio","expectancy_pct","avg_trade_days","profit_factor",
        "cagr_pct","mdd_pct","calmar","sharpe","sortino","exposure_pct","trading_days",
        "up_capture_pct","down_capture_pct","years","trades_per_year","roundtrip_turnover_per_year"
    ]:
        if k in metrics:
            print(f"{k:28s}: {metrics[k]:.6g}")

    # Also save metrics.csv
    if metrics:
        pd.DataFrame([metrics]).to_csv("metrics.csv", index=False)
        # Optional: save daily equity/returns
        if not strat_daily.empty:
            eq = (1.0 + strat_daily).cumprod()
            out = pd.DataFrame({"strategy_daily_ret": strat_daily, "strategy_equity": eq})
            out.to_csv("strategy_daily.csv", index_label="date")

if __name__ == "__main__":
    main()


Downloaded & simulated 5 tickers.
Orders -> orders.csv | Trades -> trades.csv

===== Strategy Metrics =====
trades                      : 196
win_rate_pct                : 49.4898
payoff_ratio                : 1.00861
expectancy_pct              : -0.0202155
avg_trade_days              : 8.29082
profit_factor               : 0.985344
cagr_pct                    : -3.78828
mdd_pct                     : 44.8599
calmar                      : -0.0844469
sharpe                      : -0.602801
sortino                     : -0.362484
exposure_pct                : 43.9552
trading_days                : 2680
up_capture_pct              : 1.58314
down_capture_pct            : 17.7763
years                       : 10.2697
trades_per_year             : 19.0853
roundtrip_turnover_per_year : 38.1706
